# PHOTON-Beetle Hash Function 🔐

A Python implementation of the PHOTON-Beetle cryptographic hash function, providing secure 256-bit hash generation for messages of any length.

## Overview 📝

PHOTON-Beetle is a lightweight cryptographic hash function designed for constrained environments. This implementation provides the complete PHOTON-256 permutation and hash functionality as specified in the NIST Lightweight Cryptography competition submission.

## Features ✨

- **256-bit Hash Output**: Generates secure 256-bit hash digests
- **Variable Message Length**: Supports messages from empty strings to arbitrary lengths
- **Optimized Processing**: Efficient handling of short (≤128 bits) and long messages
- **GF(2⁴) Arithmetic**: Proper finite field operations for cryptographic security
- **Complete Implementation**: Full PHOTON-256 permutation with all transformations

## Core Components 🔧

### PHOTON-256 Permutation
- **AddConstant**: Round constant addition
- **SubCells**: S-box substitution layer
- **ShiftRows**: Row shifting transformation
- **MixColumnSerial**: Column mixing using GF(2⁴) multiplication

### Hash Processing
- **Padding**: Ozs padding for proper block alignment
- **Block Processing**: Efficient handling of message blocks
- **Tag Generation**: Secure hash output generation


## Helper Functions 🛠️

### Core Functions
- `Photon_Beetle_Hash(M, r)` - Main hash function
- `PHOTON_256(X)` - PHOTON permutation
- `Hash_r(IV, D, c0, r)` - Associated data processing
- `TAG_tau(T0, tau)` - Tag generation

### Utility Functions
- `bin_to_hex(b)` - Binary to hexadecimal conversion
- `Ozs(V, r)` - Padding function
- `trunc_bitstring(V, i)` - Bit truncation
- `split_blocks(V, a)` - Block splitting

## References 📚

Based on the PHOTON-Beetle specification:
- **Paper**: [PHOTON-Beetle NIST Specification](https://csrc.nist.gov/csrc/media/Projects/Lightweight-Cryptography/documents/round-1/spec-doc/PHOTON-Beetle-spec.pdf)

In [39]:
def Ozs(V,r):
  return V + '1' + '0'*(r-len(V)-1)
def trunc_bitstring(V: str, i: int) -> str:
    """Return the most significant i bits of bitstring V."""
    if i >= len(V):
        return V
    return V[:i]
def serial(a0,a1,a2,a3,a4,a5,a6,a7):
  return [[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1],
          [a0,a1,a2,a3,a4,a5,a6,a7]
  ]
def sbox_map_bitstring(x: str) -> str:
    sbox = [
        0xC, 0x5, 0x6, 0xB,
        0x9, 0x0, 0xA, 0xD,
        0x3, 0xE, 0xF, 0x8,
        0x4, 0x7, 0x1, 0x2
    ]
    if len(x) != 4 or any(c not in "01" for c in x):
        raise ValueError("Input must be a 4-bit binary string like '1010'")

    val = int(x, 2)
    mapped = sbox[val]
    return f"{mapped:04b}"
def matrix_to_bitstring(X):
    """Convert 8x8 matrix of 4-bit strings to 256-bit string."""
    result = ""
    for i in range(8):
        for j in range(8):
            if isinstance(X[i][j], int):
                result += f"{X[i][j]:04b}"
            else:
                result += X[i][j]
    return result

def bitstring_to_matrix(bitstring):
    """Convert 256-bit string to 8x8 matrix of 4-bit strings."""
    if len(bitstring) != 256:
        raise ValueError("Input must be 256 bits")

    matrix = []
    for i in range(8):
        row = []
        for j in range(8):
            start = (i * 8 + j) * 4
            row.append(bitstring[start:start+4])
        matrix.append(row)
    return matrix

In [40]:
import math
def Add_constant(X, k):
    """AddConstant step of PHOTON-256."""
    RC = [1, 3, 7, 14, 13, 11, 6, 12, 9, 2, 5, 10]
    IC = [0, 1, 3, 7, 15, 14, 12, 8]
    X_copy = [row[:] for row in X]

    for i in range(8):
        val = int(X_copy[i][0], 2) if isinstance(X_copy[i][0], str) else X_copy[i][0]
        new_val = val ^ RC[k] ^ IC[i]
        X_copy[i][0] = f"{new_val & 0xF:04b}"

    return X_copy
def sub_cells(X):
    """SubCells step of PHOTON-256."""
    X_copy = [row[:] for row in X]
    for i in range(8):
        for j in range(8):
            X_copy[i][j] = sbox_map_bitstring(X_copy[i][j])
    return X_copy

def shift_rows(X):
    """ShiftRows step of PHOTON-256."""
    X_dash = [['' for _ in range(8)] for _ in range(8)]
    for i in range(8):
        for j in range(8):
            X_dash[i][j] = X[i][(j + i) % 8]
    return X_dash

def gf16_mult(a, b):
    """Multiply two elements in GF(2^4) with irreducible polynomial x^4 + x + 1."""
    if a == 0 or b == 0:
        return 0
    result = 0
    for i in range(4):
        if b & 1:
            result ^= a
        b >>= 1
        a <<= 1
        if a & 0x10:  # If overflow
            a ^= 0x13  # x^4 + x + 1 = 10011 = 0x13

    return result & 0xF

def Mix_Column_Serial(X):
    """MixColumnSerial step of PHOTON-256 using proper GF(2^4) arithmetic."""
    X_int = [[int(X[i][j], 2) for j in range(8)] for i in range(8)]

    # Serial matrix coefficients
    M_coeffs = [2, 4, 2, 11, 2, 8, 5, 6]

    result = [[0 for _ in range(8)] for _ in range(8)]

    # For each column
    for col in range(8):
        # Extract column
        column = [X_int[row][col] for row in range(8)]

        for _ in range(8):
            new_val = 0
            for i in range(8):
                new_val ^= gf16_mult(M_coeffs[i], column[i])

            # Shift and insert
            column = [column[1], column[2], column[3], column[4],
                     column[5], column[6], column[7], new_val]

        # Store result
        for row in range(8):
            result[row][col] = f"{column[row]:04b}"

    return result


In [41]:
def PHOTON_256(X_input):
    """PHOTON-256 permutation."""
    # Convert input to matrix format
    if isinstance(X_input, str):
        X = bitstring_to_matrix(X_input)
    else:
        X = X_input

    # 12 rounds
    for i in range(12):
        X = Add_constant(X, i)
        X = sub_cells(X)
        X = shift_rows(X)
        X = Mix_Column_Serial(X)

    # Return as bitstring
    return matrix_to_bitstring(X)

def right_rotation(X: str, i: int) -> str:
    """Right rotate bitstring X by i positions."""
    if len(X) == 0:
        return X
    i = i % len(X)
    return X[-i:] + X[:-i]

def Shuffle(S, r):
    """Shuffle function used in rho."""
    S1 = S[:r//2]
    S2 = S[r//2:r]
    return S2 + right_rotation(S1, 1)
def xor_bitstrings(a, b, target_len=256):
    """XOR two bitstrings with proper padding to target length."""
    # Pad both strings to target length
    a = a.ljust(target_len, '0')[:target_len]
    b = b.ljust(target_len, '0')[:target_len]
    
    return ''.join(str(int(x) ^ int(y)) for x, y in zip(a, b))
def rho(S, U, r):
    """Rho function for authenticated encryption."""
    shuffled = Shuffle(S, r)
    V = xor_bitstrings(trunc_bitstring(shuffled, len(U)), U)
    S_new = xor_bitstrings(S, Ozs(U, r))
    return S_new, V

def rho_inverse(S, V, r):
    """Inverse rho function for decryption."""
    shuffled = Shuffle(S, r)
    U = xor_bitstrings(trunc_bitstring(shuffled, len(V)), V)
    S_new = xor_bitstrings(S, Ozs(U, r))
    return S_new, U

def split_blocks(V: str, a: int) -> list:
    """Split bitstring V into blocks of size a."""
    return [V[i:i+a] for i in range(0, len(V), a)]

In [42]:

def Hash_r(IV, D, c0, r):
    """Hash function for processing associated data with proper 256-bit handling."""
    d_temp = Ozs(D, r)
    D_blocks = split_blocks(d_temp, r)

    current_IV = IV
    # Ensure IV is exactly 256 bits
    if len(current_IV) != 256:
        current_IV = current_IV.ljust(256, '0')[:256]

    for Di in D_blocks:
        permutation_output = PHOTON_256(current_IV)
        Y = trunc_bitstring(permutation_output, r)
        Z = permutation_output[r:]
        W = xor_bitstrings(Y, Di)
        # Ensure the concatenated result is exactly 256 bits
        current_IV = (W + Z).ljust(256, '0')[:256]

    c0_bits = f"{c0:08b}"  
    c0_padded = "0" * (256 - 8) + c0_bits  
    current_IV = xor_bitstrings(current_IV, c0_padded)

    return current_IV

def TAG_tau(T0, tau):
    """Generate tag of tau bits."""
    T = [T0]
    num_calls = math.ceil(tau / 128)
    for i in range(1, num_calls):
        T.append(PHOTON_256(T[i-1]))
    result = ""
    for i in range(num_calls):
        result += trunc_bitstring(T[i], min(128, tau - i * 128))

    return trunc_bitstring(result, tau)

In [43]:
def Photon_Beetle_Hash(M, r):
    """PHOTON-Beetle Hash function with proper 256-bit handling."""
    if M == "":
        IV = "0" * 255 + "1"  # 256 bits with last bit set to 1
        T = TAG_tau(IV, 256)
        return T

    if len(M) <= 128:
        c0 = 1 if len(M) < 128 else 2
        padded_M = Ozs(M, 128) 
        IV = (padded_M + "0" * 128).ljust(256, '0')[:256]  # Ensure 256 bits
        c0_bits = "0" * (256 - 8) + f"{c0:08b}"  
        iv_xor_c0 = xor_bitstrings(IV, c0_bits, 256)

        T = TAG_tau(iv_xor_c0, 256)
        return T
    
    # For messages > 128 bits
    M1 = M[:128]  # First 128 bits
    M_prime = M[128:]  # Remaining bits
    c0 = 1 if len(M_prime) % r == 0 else 2
    IV = (M1 + "0" * 128).ljust(256, '0')[:256]  # Ensure exactly 256 bits
    IV = Hash_r(IV, M_prime, c0, r)
    T = TAG_tau(IV, 256)
    return T

In [44]:
def bin_to_hex(b: str) -> str:
    """Convert a binary string to hexadecimal string."""
    b = b.zfill((len(b) + 3) // 4 * 4)
    return hex(int(b, 2))[2:]

In [47]:

def test_hash_function():
    """Test PHOTON-Beetle Hash function with fixes."""
    print("\n=== Testing PHOTON-Beetle Hash Function (Fixed) ===")
    r = 32
    
    print("Test 1: Empty message")
    try:
        T1 = Photon_Beetle_Hash("", r)
        print(f"Hash of empty string: {bin_to_hex(T1)} (length: {len(T1)})")
    except Exception as e:
        print(f"Error with empty message: {e}")

    # Short message (< 128 bits)
    print("Test 2: Short message")
    try:
        M2 = "11101011001110101"
        T2 = Photon_Beetle_Hash(M2, r)
        print(f"Hash of '{M2}': {bin_to_hex(T2)} (length: {len(T2)})")
    except Exception as e:
        print(f"Error with short message: {e}")

    # Exactly 128-bit message
    print("Test 3: Exactly 128-bit message")
    try:
        M3 = "0" * 125 + "111"    
        T3 = Photon_Beetle_Hash(M3, r)
        print(f"Hash of 128-bit message: {bin_to_hex(T3)} (length: {len(T3)})")
    except Exception as e:
        print(f"Error with 128-bit message: {e}")

    # Long message (> 128 bits)
    print("Test 4: Long message")
    try:
        M4 = "1000010100100011010001010110011110001001101010111100110111101111" * 600
        print(f"Message length: {len(M4)}")
        T4 = Photon_Beetle_Hash(M4, r)
        print(f"Hash of long message: {bin_to_hex(T4)} (length: {len(T4)})")
    except Exception as e:
        print(f"Error with long message: {e}")

test_hash_function()


=== Testing PHOTON-Beetle Hash Function (Fixed) ===
Test 1: Empty message
Hash of empty string: 49b0816d73ec81443e6498b2574abe24 (length: 256)
Test 2: Short message
Hash of '11101011001110101': eb3ac0000000000000000000000000002dff383de5eae3e1d978dbb0ea7d301a (length: 256)
Test 3: Exactly 128-bit message
Hash of 128-bit message: 72e3e23744b3d0eca3da1e9d498b892cd (length: 256)
Test 4: Long message
Message length: 38400
Hash of long message: f0bc03db000000000000000000000000d09178e1cf835f1b5c3964136f4421af (length: 256)
